# MD5-light-Algorithm

In [ ]:
import math
import struct

class Hash_value:
    def __init__(self, hashvalue_hexa):
        self.hashvalue_hexa = hashvalue_hexa
        self.hashvalue_bits = bin(int(hashvalue_hexa, 16))[2:].zfill(len(hashvalue_hexa) * 4)
        self.hashvalue_integer =  int(hashvalue_hexa, 16)
        self.hashvalue_bytes = bytes.fromhex(self.hashvalue_hexa)

class MD5_Hash:
    # Constructor (__init__)
    def __init__(self, type = "regular"):
        # Set all Variables and Constants dependend on wordsize_bit
        if (type == "regular"):
            self.wordsize_bit = 32
            self.bits_length_rep = 'Q' # 64 bit / 8 Byte
            self.padding_length = 64 - 8
        elif (type == "light"):
            self.wordsize_bit = 8
            self.bits_length_rep = 'H' # 16 bit / 2 Byte
            self.padding_length = 16 - 2
        else: raise ValueError("hash type must be either light or regular")
        self.wordsize_byte = int(self.wordsize_bit / 8)
        self.blocksize_byte = self.wordsize_byte * 16
        self.hexa = (1 << (self.wordsize_byte * 8)) - 1
        a0, b0, c0, d0 = 0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476
        self.Ks = [int(abs(math.sin(i + 1)) * (2**self.wordsize_bit)) & self.hexa for i in range(64)]
        self.Ss = list(map(lambda s: s % self.wordsize_bit,[7, 12, 17, 22] * 4 + [5, 9, 14, 20] * 4 + [4, 11, 16, 23] * 4 + [6, 10, 15, 21] * 4))
        self.a0,self.b0,self.c0,self.d0 = map(lambda x: int(hex(x)[2:].upper()[:(self.wordsize_byte * 2)],16), [a0, b0, c0, d0])
        self.hashlength_bit = 4* self.wordsize_bit
        self.hashlength_byte = 4* self.wordsize_byte

    #convert Type
    def convertType(self,m):
        if isinstance(m, bytes):
            return m
        # Make sure m is converted to byte value
        if isinstance(m, str) and all(c in '01' for c in m):  # If input is a bit string
            byte_length = (len(m) + 7) // 8  # Compute required byte length
            return int(m, 2).to_bytes(byte_length, byteorder="big")
        elif isinstance(m, str):  # If input is a message (string)
            return m.encode('utf-8') # bytes
        else:
            raise ValueError("Input must be either a message (string) or a valid bit string.")

    #Padding Function
    def pad(self,m):
        bytes = self.convertType(m)
        original_length_bits = len(bytes) * 8
        bytes += b'\x80' #appends 10000000
        while len(bytes)%(self.blocksize_byte)!=self.padding_length:
            bytes += b'\x00' #appends 00000000
        bytes += struct.pack(f'<{self.bits_length_rep}', original_length_bits) # appends 16 bit /64 bit representation in litte-endian-format of length
        return bytes

    # Rotation function (rotate left)
    def rotate_left(self, x, n):
        # Perform a left rotation on a 32-bit integer 'x' by 'n' positions
        x &= self.hexa
        return ((x << n) | (x >> (self.wordsize_bit - n))) & self.hexa

    #digest
    def digestABCD(self, A,B,C,D):
        raw = sum(value << (self.wordsize_bit * i) for i, value in enumerate([A,B,C,D])).to_bytes(16, byteorder='little')
        hashvalue_hexa = '{:0{width}x}'.format(int.from_bytes(raw[:self.hashlength_byte], byteorder='big'), width = self.wordsize_bit)
        hv = Hash_value(hashvalue_hexa)
        return Hash_value(hashvalue_hexa)

    #Hash
    def processblock(self, block, inits):
        A, B, C, D = inits
        for i in range(64):
            if i <= 15:
                f = (B & C) | (~B & D)
                g = i
            elif i <= 31:
                f = (B & D) | (C & ~D)
                g = (5 * i + 1) % 16
            elif i <= 47:
                f = B ^ C ^ D
                g = (3 * i + 5) % 16
            elif i <= 63:
                f = C ^ (B | ~D)
                g = (7 * i) % 16

            # Calculate the temporary values
            temp = D
            D = C
            C = B
            M = int.from_bytes(block[self.wordsize_byte * g : self.wordsize_byte * g + self.wordsize_byte], byteorder='little')
            B = (B + self.rotate_left(A + f + self.Ks[i] + M, self.Ss[i])) & self.hexa
            A = temp
        return([A,B,C,D])

    def digest(self,m):
        blocks = self.pad(m)
        # Process each block
        A_final,B_final,C_final,D_final = self.a0, self.b0, self.c0, self.d0
        for offset in range(0, len(blocks), self.blocksize_byte):
            block = blocks[ offset : offset + self.blocksize_byte]

            #Compute Hash
            A,B,C,D = self.processblock(block, [A_final,B_final,C_final,D_final])
            A_final, B_final, C_final, D_final = (A_final + A) & self.hexa, (B_final + B) & self.hexa, (C_final + C) & self.hexa, (D_final + D) & self.hexa
        return self.digestABCD(A_final, B_final, C_final, D_final)

#check correctness by comparing with md5 library
import hashlib
bytedata = "Hello World!".encode('utf-8')
md5_hash = hashlib.md5(bytedata).digest()
computed_hash = MD5_Hash().digest(bytedata).hashvalue_bytes
if (md5_hash != computed_hash):
    print(f"Correct hash:  {md5_hash}")
    print(f"Computed hash: {computed_hash}")

def md5_light(input_data: str) -> str:
    # use MD5 - light to calculate hash
    Hash = MD5_Hash('light')
    hashvalue = Hash.digest(input_data)
    return hashvalue.hashvalue_bytes

md5_light("hello")

b'\x1bS|\x9e'

# HMAC

## HMAC Construction:
$$
 \text{HMAC}(K,M) = H((K'⊕ opad)||H(K'⊕ ipad)||M))
$$
where:
- **$H$:** is your custom 32-bit hash function.
- **$K$:** is the secret key
- **$M$:** is the message
- **$K'$:** is a key padded or truncated to the block size
- **$⊕$:** represents XOR
- **$opad$:** outer pad is 0x5c repeated to the block size
- **$ipad$:** inner pad is 0x36 repeated to the block size

### Steps:
1. Ensure the key is at most the block size of your hash function. If longer, hash it.
2. If the Key is shorter, pad it with zero bytes
3. Compute the inner hash: $$H(K'⊕ ipad)||M) $$
4. Compute the outer hash: $$H(K'⊕ opad)||\text{inner hash}) $$

In [ ]:
import hashlib

def hmac(key: bytes, message: bytes, hash_func = md5_light, block_size=16) -> bytes:
    # Ensure the key is the right length
    if len(key) > block_size:
        key = hash_func(key)  # Hash the key if too long
    if len(key) < block_size:
        key = key.ljust(block_size, b'\x00')  # Pad with zeros if too short

    # Define the opad and ipad
    opad = bytes((x ^ 0x5C) for x in key)
    ipad = bytes((x ^ 0x36) for x in key)

    # Inner hash
    inner_hash = hash_func(ipad + message)

    # Outer hash
    return hash_func(opad + inner_hash)

# Example Usage
key = b'secretkey'
message = b'hello'
hmac_result = hmac(key, message)
print("HMAC:", hmac_result)

HMAC: b'\xd1\x9b\xee\x84'


# Training Data

## Generate Data

In [ ]:
import numpy as np
import os

# Generate random bitstring messages
def random_bitstring(length=128):
    return ''.join(np.random.choice(['0', '1'], size=length))

# === PARAMETERS ===
num_samples = 10000
keylength = 4 # 4 byte
dataset = []
# === DATASET GENERATION ===
for _ in range(num_samples):

    # Generate random 4 byte key
    k1 = os.urandom(keylength)
    k2 = k1 if np.random.rand() > 0.5 else os.urandom(keylength)

    # Generate m, m2 pair with desired differential
    m1 = bytearray(os.urandom(11))
    m2 = m1[:]
    m2[8] = (m1[8] + 0x80) % 256

    #compute HMACs
    hmac1 = hmac(k1,m1)
    hmac2 = hmac(k2,m2)

    #Label
    label = 1 if k1 == k2 else 0
    dataset.append((hmac1,hmac2,label))


## Access Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Path to the CSV file on Google Drive
file_path = '/content/drive/MyDrive/Colab Notebooks/HMACs/'
filename = f'{file_path}dataset_1million.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Upload Data

In [ ]:
import csv
# === SAVE TO CSV ===
with open(filename, mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["hmac(m1)","hmac(m2)","sameKey?"])
    for row in dataset:
      writer.writerow([row[0],row[1],row[2]])

## Download Data

In [ ]:
import pandas as pd
import ast
# Load the dataset
df = pd.read_csv(filename)

data = df.to_numpy()
def bytes_to_bitstring(byte_string):
    byte_data = bytes(ast.literal_eval(byte_string))
    return ''.join(f'{byte:08b}' for byte in byte_data)
dataset = [(bytes_to_bitstring(row[0]),bytes_to_bitstring(row[0]), int(row[2])) for row in data]

## Prepare Data

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
# Convert dataset to arrays
X = np.array([list(row[0]) + list(row[1]) for row in dataset])
y = np.array([row[2] for row in dataset])

X = X.astype(np.float32)
y = y.astype(np.float32)
print(f"X shape: {X.shape}, dtype: {X.dtype}")  # Should be (num_samples, 64)
print(f"y shape: {y.shape}, dtype: {y.dtype}")  # Should be (num_samples,)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X shape: (1000000, 64), dtype: float32
y shape: (1000000,), dtype: float32


# Define and Train the Neural Netwok Model (CNN)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import time
import tracemalloc
from colorama import Fore, Style

# === Define the Network ===

# Hyperparameters to use
num_filters = 64
kernel_size = 2
num_conv_layers = 2
dense_units = 96
learning_rate =  0.0002
batch_size = 352
activation = "leaky_relu"

#Leaky relu
if activation == "leaky_relu":
    activation_func = layers.LeakyReLU()
else:
  activation_func = activation

 # === Build CNN Model ===
model = keras.Sequential()
model.add(layers.Input(shape=(64, 1)))
for _ in range(num_conv_layers):
  model.add(layers.Conv1D(filters=num_filters, kernel_size=kernel_size, activation=activation_func, strides=1, padding='same'))
  model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(dense_units, activation=activation_func))
model.add(layers.Dense(dense_units, activation=activation_func))
model.add(layers.Dense(1, activation="sigmoid"))

# === Compile Model ===
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)                    │ (None, 64, 64)              │             192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_4 (MaxPooling1D)       │ (None, 32, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_5 (Conv1D)                    │ (None, 32, 64)              │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_5 (MaxPooling1D)       │ (None, 16, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 96)                  │          98,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 96)                  │           9,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              97 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 116,257 (454.13 KB)

 Trainable params: 116,257 (454.13 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# === Track Time and Memory Usage ===
tracemalloc.start()
start_time = time.time()

# === Train model ===
early_stopping = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=50, batch_size=batch_size, validation_data=(X_test, y_test), callbacks = [early_stopping])

# === Output Time taken and memory used ===
_, peak = tracemalloc.get_traced_memory()  # Memory in bytes
tracemalloc.stop() # Stop tracking
elapsed_time = time.time() - start_time
print(Fore.RED + f"Time taken: {(elapsed_time / 60):.2f}min" + Style.RESET_ALL)
print(Fore.GREEN + f"Memory Used: {peak:.2f} MB" + Style.RESET_ALL)

Epoch 1/50
2557/2557 ━━━━━━━━━━━━━━━━━━━━ 161s 60ms/step - accuracy: 0.5012 - loss: 0.6933 - val_accuracy: 0.4970 - val_loss: 0.6933
Epoch 2/50
2557/2557 ━━━━━━━━━━━━━━━━━━━━ 202s 61ms/step - accuracy: 0.5007 - loss: 0.6932 - val_accuracy: 0.5021 - val_loss: 0.6931
Epoch 3/50
2557/2557 ━━━━━━━━━━━━━━━━━━━━ 203s 61ms/step - accuracy: 0.5031 - loss: 0.6931 - val_accuracy: 0.5005 - val_loss: 0.6932
Epoch 4/50
2557/2557 ━━━━━━━━━━━━━━━━━━━━ 155s 61ms/step - accuracy: 0.5044 - loss: 0.6931 - val_accuracy: 0.4979 - val_loss: 0.6933
Epoch 5/50
2557/2557 ━━━━━━━━━━━━━━━━━━━━ 155s 60ms/step - accuracy: 0.5073 - loss: 0.6929 - val_accuracy: 0.4974 - val_loss: 0.6933
Epoch 6/50
2557/2557 ━━━━━━━━━━━━━━━━━━━━ 202s 61ms/step - accuracy: 0.5106 - loss: 0.6928 - val_accuracy: 0.4999 - val_loss: 0.6934
Epoch 7/50
2557/2557 ━━━━━━━━━━━━━━━━━━━━ 199s 59ms/step - accuracy: 0.5132 - loss: 0.6926 - val_accuracy: 0.5008 - val_loss: 0.6934
Time taken: 22.13min
Memory Used: 242716341.00 MB


# Optuna

In [ ]:
import optuna
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import time
import tracemalloc
from colorama import Fore, Style

historys = []
params_ = []
trial_times = []
trial_memory = []

# === Define the Objective Function for Optuna ===
def objective(trial):
    # === Track Time and Memory Usage ===
    tracemalloc.start()
    start_time = time.time()

    # Hyperparameters to tune
    num_filters = trial.suggest_int("num_filters", 16, 64, step=16)
    kernel_size = trial.suggest_int("kernel_size", 1, 3, step=1)
    num_conv_layers = trial.suggest_int("num_conv_layers", 2,4, step=1)
    dense_units = trial.suggest_int("dense_units", 32, 128, step=32)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_int("batch_size", 32, 1024, step=32)
    activation = trial.suggest_categorical("activation", ["relu", "leaky_relu","selu"])

    #Leaky relu
    if activation == "leaky_relu":
        activation_func = layers.LeakyReLU()
    else:
      activation_func = activation

    # === Build CNN Model ===
    model = keras.Sequential()
    model.add(layers.Input(shape=(64, 1)))

    for _ in range(num_conv_layers):
      model.add(layers.Conv1D(filters=num_filters, kernel_size=kernel_size, activation=activation_func, strides=1, padding='same'))
      model.add(layers.MaxPooling1D(pool_size=2))

    model.add(layers.Flatten())
    model.add(layers.Dense(dense_units, activation=activation_func))
    model.add(layers.Dense(dense_units, activation=activation_func))
    model.add(layers.Dense(1, activation="sigmoid"))

    # Compile model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss="binary_crossentropy",
                  metrics=["accuracy"])

    # Train model
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        epochs=50, batch_size=batch_size, verbose=0, callbacks=[early_stopping])

    historys.append(history)
    params_.append([num_filters, kernel_size, num_conv_layers, dense_units, learning_rate, batch_size, activation])

    _, peak = tracemalloc.get_traced_memory()  # Memory in bytes
    tracemalloc.stop() # Stop tracking
    elapsed_time = time.time() - start_time
    trial_times.append(elapsed_time)
    trial_memory.append(peak / 1e6)
    # Evaluate accuracy
    val_accuracy = history.history["val_accuracy"][-1]
    return val_accuracy

# === Run Optuna Optimization ===
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# === Print best hyperparameters ===
print(Fore.CYAN + f"\nBest Hyperparameters: {study.best_params}" + Style.RESET_ALL)

# === Output Time taken and memory used ===
print(Fore.RED + f"Total Time taken: {(sum(trial_times)/60):.2f}min" + Style.RESET_ALL)
print(Fore.YELLOW + f"Average Time Per Trial: {sum(trial_times)/len(trial_times):.2f}s" + Style.RESET_ALL)
print(Fore.GREEN + f"Average Memory Used: {sum(trial_memory)/len(trial_memory):.2f} MB" + Style.RESET_ALL)


[I 2025-02-08 01:56:28,053] A new study created in memory with name: no-name-3384d107-0849-4a54-96e3-3d9254690752
[I 2025-02-08 01:58:14,969] Trial 0 finished with value: 0.49869999289512634 and parameters: {'num_filters': 32, 'kernel_size': 3, 'num_conv_layers': 2, 'dense_units': 64, 'learning_rate': 0.0011072850070923531, 'batch_size': 896, 'activation': 'relu'}. Best is trial 0 with value: 0.49869999289512634.
[I 2025-02-08 02:00:34,694] Trial 1 finished with value: 0.4959999918937683 and parameters: {'num_filters': 32, 'kernel_size': 3, 'num_conv_layers': 3, 'dense_units': 64, 'learning_rate': 0.000263588948164315, 'batch_size': 992, 'activation': 'selu'}. Best is trial 0 with value: 0.49869999289512634.
[I 2025-02-08 02:04:01,481] Trial 2 finished with value: 0.5073999762535095 and parameters: {'num_filters': 64, 'kernel_size': 2, 'num_conv_layers': 2, 'dense_units': 96, 'learning_rate': 0.00019834734763826554, 'batch_size': 352, 'activation': 'leaky_relu'}. Best is trial 2 with v


Best Hyperparameters: {'num_filters': 64, 'kernel_size': 2, 'num_conv_layers': 2, 'dense_units': 96, 'learning_rate': 0.00019834734763826554, 'batch_size': 352, 'activation': 'leaky_relu'}
Total Time taken: 77.43min
Average Time Per Trial: 232.30s
Average Memory Used: 30.84 MB


# install package

In [ ]:
!pip install colorama


In [ ]:
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.9 MB/s eta 0:00:00
